In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import time

In [2]:
def splitDateInColumns(df):
    for index, row in df.iterrows():
        date_object = row["Date"]
        df.at[index, 'Day'] = date_object.day
        df.at[index, 'Month'] = date_object.month
        df.at[index, 'Year'] = date_object.year

    return df

In [3]:
# Solution from : https://stackoverflow.com/questions/32854677/how-to-deal-with-multiple-date-string-formats-in-a-python-series
def timeToSeconds(timestr):
    for fmt in ("%M:%S.%f", "%S.%f", "%S:%f", "%M:%S:%f"):
        try:
            t = datetime.strptime(timestr, fmt)
            seconds = (t.minute * 60) + t.second + (t.microsecond * 1e-6)
            return round(seconds)
            break
        except ValueError:
            pass

In [4]:
def createTeams(df):
    for index, row in df.iterrows():
        person1 = row["Person 1"].upper()
        person2 = row["Person 2"].upper()
        team = (person1 + "_" + person2).replace(" ","")
        team = team.replace(",","")
        df.at[index,"Team"]  = team
    return df
    

In [5]:
def cleanColumns(df):
    #df.info()
    df = df.dropna(subset=['Date', 'Person 1', 'Person 2','split time 1','split time 2','split time 3', 'Total Time'])

    for index,row in df.iterrows():
        df.at[index,"Date"] = row["Date"] + " " + row["Time"]

    df['Date'] = pd.to_datetime(df['Date'])

    df.set_index("Date")
    df = df.sort_index()
    return df

In [7]:
def createIDs(df):
    df['Competition_ID'] = df.groupby(["Year","Place"]).grouper.label_info
    df['Race_ID'] = df.groupby(["Date"]).grouper.label_info
    df['Team_ID'] = df.groupby(["Team"]).grouper.label_info
    return df

In [8]:
def addTimeToDateIndex(df):
    for index,row in df.iterrows():
        timeObj = datetime.strptime(row["Time"],'%H:%M')
        df.at[index,"Date"] = datetime.datetime.combine(row["Date"],timeObj)
    return df 

In [9]:
def convertTimes(df):
    for index, row in df.iterrows():
        df.at[index, "split time 1"] = timeToSeconds(row["split time 1"])
        df.at[index, "split time 2"] = timeToSeconds(row["split time 2"])
        df.at[index, "split time 3"] = timeToSeconds(row["split time 3"])
        df.at[index, "Total Time"] = timeToSeconds(row["Total Time"])
    return df


In [10]:
def createCompetitionColumns(df):
    df = pd.get_dummies(df,columns=['Competiton Type','Competition Round'], dtype=bool)
    return df

In [11]:
def create_DeltaTimes(df):
   for index, row in df.iterrows():
      df.at[index,"D0_250"] = row["split time 1"]
      df.at[index,"D250_500"] = pd.to_numeric(row["split time 2"])-pd.to_numeric(row["split time 1"])
      df.at[index,"D500_750"] = pd.to_numeric(row["split time 3"])-pd.to_numeric(row["split time 2"])
      df.at[index,"D750_1000"] = pd.to_numeric(row["Total Time"])-pd.to_numeric(row["split time 3"])
   return df


In [12]:
def createPb_TimecurrentSeason(df):
    for index, row in df.iterrows():
        #Verkrijg alle resultaten voor deze datum en van hetzelfde jaar 
        results = df[(df["Year"] == row["Year"]) & (df["Date"] < row["Date"]) & (df["Team"]==row["Team"])].sort_values("Total Time")
        if(results.shape[0] > 0):
            bestTime = results["Total Time"].iloc[0]
            df.at[index,"BestSeasonTime"] = bestTime
        else: 
            bestTime = row["Total Time"]
            df.at[index,"BestSeasonTime"] = bestTime
    return df
    

In [13]:
#Shows how many races a team has paddled together before current race
def createCompetitionTotal(df):
    for index, row in df.iterrows():
        #Get all the results of a Team before current Race
        results = df[(df["Date"] < row["Date"]) & (df["Team"]==row["Team"])]
        #Count 1 Competition with multiple races as 1
        uniqueCount = results.groupby(['Year','Place']).ngroups
        df.at[index,"TotalRacesTogether"] = uniqueCount
    return df

In [14]:
#Get the count of A finals before the current Race
def getTotalAFinalsTeam(df):
    for index, row in df.iterrows():
        #Get all The races before this race of one team where the competition round was an A Final
        results = df[(df["Date"] < row["Date"]) & (df["Team"]==row["Team"]) & (df["Competition Round_FINAL A"] == True) ]
        #Count 1 Competition with multiple races as 1
        uniqueCount = results.groupby(['Year','Place']).ngroups
        df.at[index,"TotalAFinalsTogether"] = uniqueCount
    return df

In [15]:
#Get the count of Semi finals before the current Race
def getTotalSemiFinalsTeam(df):
    for index, row in df.iterrows():
        #Get all The races before this race of one team where the competition round was an Semi Final
        results = df[(df["Date"] < row["Date"]) & (df["Team"]==row["Team"]) & (df["Competition Round_SEMIFINAL"] == True) ]
        #Count 1 Competition with multiple races as 1
        uniqueCount = results.groupby(['Year','Place']).ngroups
        df.at[index,"TotalSemiFinalsTogether"] = uniqueCount
    return df

In [16]:
def create_Reached_FinalA(df):
    #Loop over elke row
    for index, row in df.iterrows():
        #check of ploeg in row i in de a finale komt
        dataCount = df[(df["Competition Round_FINAL A"] == True) & (df["Year"] == row["Year"]) & (df["Place"] == row["Place"]) & (df["Team"] == row["Team"])].shape[0]
        #als ploeg in de a finale geraakt
        if(dataCount >= 1):
            df.at[index,"Reached_FINAL_A"] = True
        else: 
            df.at[index,"Reached_FINAL_A"] = False        
    return df
            

In [17]:
def create_Reached_Semi(df):
    #Loop over elke row
    for index, row in df.iterrows():
        #check of ploeg in row i in de Semi komt
        dataCount = df.loc[(df["Competition Round_SEMIFINAL"] == True) & (df["Year"] == row["Year"]) & (df["Place"] == row["Place"]) & (df["Team"] == row["Team"])].shape[0]
        #als ploeg in de a finale geraakt
        if(dataCount >= 1):
            df.at[index,"Reached_SEMI"] = True
        else: 
            df.at[index,"Reached_SEMI"] = False        
    return df

In [18]:
def create_count_FinalA_currentSeason(df):
    for index, row in df.iterrows():
        #Verkrijg alle resultaten voor deze datum en van hetzelfde jaar 
        results = df[(df["Year"] == row["Year"]) & (df["Date"] < row["Date"]) & (df["Team"]==row["Team"]) & (df["Competition Round_FINAL A"] == True)].shape[0]
        df.at[index,"FinalA_count_Season"] = results
    return df

In [19]:


def rankeRaces(df):
    Results = pd.DataFrame(columns=["Race_ID","TT1","TT2","TT3","TT4","TT5","TT6","TT7","TT8","TT9"])
    racesGrouped = df.groupby("Race_ID")

    for name, group in racesGrouped:
        race = group.reset_index()
        race = race.sort_values(by="Total Time")["Total Time"].to_numpy()
        race = np.append(race,np.full(9-race.size,-1))

        Results = Results.append({"Race_ID":name,"TT1":race[0],"TT2":race[1],"TT3":race[2],"TT4":race[3],"TT5":race[4],"TT6":race[5],"TT7":race[6],"TT8":race[7],"TT9":race[8]},ignore_index=True)
    
    df = pd.merge(df,Results,on="Race_ID")
    return df

def rankeRaces250(df):
    Results = pd.DataFrame(columns=["Race_ID","T1_250","T2_250","T3_250","T4_250","T5_250","T6_250","T7_250","T8_250","T9_250"])
    racesGrouped = df.groupby("Race_ID")

    for name, group in racesGrouped:
        race = group.reset_index()
        race = race.sort_values(by="split time 1")["split time 1"].to_numpy()
        race = np.append(race,np.full(9-race.size,-1))

        Results = Results.append({"Race_ID":name,"T1_250":race[0],"T2_250":race[1],"T3_250":race[2],"T4_250":race[3],"T5_250":race[4],"T6_250":race[5],"T7_250":race[6],"T8_250":race[7],"T9_250":race[8]},ignore_index=True)
    
    df = pd.merge(df,Results,on="Race_ID")
    return df

def rankeRaces500(df):
    Results = pd.DataFrame(columns=["Race_ID","T1_500","T2_500","T3_500","T4_500","T5_500","T6_500","T7_500","T8_500","T9_500"])
    racesGrouped = df.groupby("Race_ID")

    for name, group in racesGrouped:
        race = group.reset_index()
        race = race.sort_values(by="split time 1")["split time 2"].to_numpy()
        race = np.append(race,np.full(9-race.size,-1))

        Results = Results.append({"Race_ID":name,"T1_500":race[0],"T2_500":race[1],"T3_500":race[2],"T4_500":race[3],"T5_500":race[4],"T6_500":race[5],"T7_500":race[6],"T8_500":race[7],"T9_500":race[8]},ignore_index=True)
    
    df = pd.merge(df,Results,on="Race_ID")
    return df

def rankeRaces750(df):
    Results = pd.DataFrame(columns=["Race_ID","T1_750","T2_750","T3_750","T4_750","T5_750","T6_750","T7_750","T8_750","T9_750"])
    racesGrouped = df.groupby("Race_ID")

    for name, group in racesGrouped:
        race = group.reset_index()
        race = race.sort_values(by="split time 1")["split time 3"].to_numpy()
        race = np.append(race,np.full(9-race.size,-1))

        Results = Results.append({"Race_ID":name,"T1_750":race[0],"T2_750":race[1],"T3_750":race[2],"T4_750":race[3],"T5_750":race[4],"T6_750":race[5],"T7_750":race[6],"T8_750":race[7],"T9_750":race[8]},ignore_index=True)
    
    df = pd.merge(df,Results,on="Race_ID")
    return df

In [20]:
#Verkrijg de gemiddelde tijd die er in een race is gevaren
def getAverageTimeofRace(df):
    dfc = df.groupby("Date")["Total Time"].mean().to_frame("avgTotalTime").reset_index()
    

In [21]:
def dropColumns(df):
    df = df.drop(columns=["Person 1", "Person 2","Wind Speed", "Wind Direction","Sex","Category"],axis=1)
    return df

In [22]:
#Bereken de gemiddelde Totaal Tijd van de eerste 3 boten in een race
def createAVGTIMETOP3(df):
    #behoud enkel de boten die in de top 3 zijn geraakt per race
    filterTop3= df[(df["Final Rank"] < 4)]
    #Tijd naar nummer converteren
    filterTop3["Total Time"] = pd.to_numeric(filterTop3["Total Time"])
    #Groepeer per race en neem het gemiddelde van deze groep(race)
    groupbyRace = filterTop3.groupby("Date")["Total Time"].mean().to_frame(name="Top3Avg").reset_index()
    #Merge  gemiddelden met dataset
    df = pd.merge(df,groupbyRace,how="outer")
    return df

In [23]:
#Creeer de snelste tijd in de Heats per Competitie
def createFastestHeatTimeInCompetition(df):
    #behoud enkel de heats
    filterHeat = df[(df["Competition Round_HEAT"] == True)]
    #groepeer per Competitie(Plaat,Jaar) en neem de kleinste totaal tijd
    raceGroups = filterHeat.groupby(["Place","Year"])["Total Time"].min().to_frame(name="N1THeatCompetition").reset_index()
    #Merge de data set
    df = pd.merge(df,raceGroups,on=["Place","Year"])
    return df

#Creeer de snelste tijd in de Semi Finals per Competitie
def createFastestSemiTimeInCompetition(df):
    #behoud enkel de semis
    filterSemi = df[(df["Competition Round_SEMIFINAL"] == True)]
    #groepeer per Competitie(Plaat,Jaar) en neem de kleinste totaal tijd
    raceGroups = filterSemi.groupby(["Place","Year"])["Total Time"].min().to_frame(name="N1TSemiCompetition").reset_index()
    #Merge de data set
    df = pd.merge(df,raceGroups,on=["Place","Year"])
    return df

#Creeer de snelste tijd in de A finale per Competitie
def createFastestFinalTimeInCompetition(df):
    #behoud enkel de A Finales
    filterFinal = df[(df["Competition Round_FINAL A"] == True)]
    #groepeer per Competitie(Plaat,Jaar) en neem de kleinste totaal tijd
    raceGroups = filterFinal.groupby(["Place","Year"])["Total Time"].min().to_frame(name="N1TFinalACompetition").reset_index()
    #Merge de data set
    df = pd.merge(df,raceGroups,on=["Place","Year"])
    return df
    
def createFastestTimeInCompetition(df):
    raceGroups = df.groupby("Competition_ID")["Total Time"].min().to_frame(name="N1TCompetition").reset_index()
    #Merge de data set
    df = pd.merge(df,raceGroups,on="Competition_ID")
    return df

In [24]:
# Creeer het gemiddelde van de snelste 3 tijden in de heats per Competitie
def createAvgTop3HeatTimeInCompetition(df):
    Averages = pd.DataFrame(columns=["Competition_ID","HeatTop3Avg"])
    df["Total Time"] = pd.to_numeric(df["Total Time"])
    HeatTimes = df[(df["Competition Round_HEAT"] == True)].copy()
    competitionGroups = HeatTimes.groupby("Competition_ID")
    for name, group in competitionGroups:
        Averages = Averages.append({"Competition_ID":str(name),"HeatTop3Avg":group.nsmallest(3,"Total Time")["Total Time"].mean()},ignore_index=True)

    Averages["Competition_ID"] = Averages["Competition_ID"].astype('int64')
    df = pd.merge(df,Averages,on="Competition_ID")
    return df

# Creeer het gemiddelde van de snelste 3 tijden in de SemiFinals per Competitie
def createAvgTop3SemiTimeInCompetition(df):
    Averages2 = pd.DataFrame(columns=["Competition_ID","SemiTop3Avg"])
    df["Total Time"] = pd.to_numeric(df["Total Time"])
    SemiTimes = df[(df["Competition Round_SEMIFINAL"] == True)].copy()
    competitionGroups2 = SemiTimes.groupby("Competition_ID")
    for name, group in competitionGroups2:
        Averages2 = Averages2.append({"Competition_ID":str(name),"SemiTop3Avg":group.nsmallest(3,"Total Time")["Total Time"].mean()},ignore_index=True)

    Averages2["Competition_ID"] = Averages2["Competition_ID"].astype('int64')
    df = pd.merge(df,Averages2,on="Competition_ID")
    return df

# Creeer het gemiddelde van de snelste 9 tijden in de SemiFinals per Competitie
def createAvgTop9SemiTimeInCompetition(df):
    Averages = pd.DataFrame(columns=["Competition_ID","SemiTop9Avg"])
    df["Total Time"] = pd.to_numeric(df["Total Time"])
    SemiTimes = df[(df["Competition Round_SEMIFINAL"] == True)].copy()
    competitionGroups = SemiTimes.groupby("Competition_ID")
    for name, group in competitionGroups:
        Averages = Averages.append({"Competition_ID":str(name),"SemiTop9Avg":group.nsmallest(9,"Total Time")["Total Time"].mean()},ignore_index=True)

    Averages["Competition_ID"] = Averages["Competition_ID"].astype('int64')
    df = pd.merge(df,Averages,on="Competition_ID")
    return df

# Creeer het gemiddelde van de snelste 3 tijden in de heats per Competitie
def createAvgTop9HeatTimeInCompetition(df):
    Averages = pd.DataFrame(columns=["Competition_ID","HeatTop9Avg"])
    df["Total Time"] = pd.to_numeric(df["Total Time"])
    HeatTimes = df[(df["Competition Round_HEAT"] == True)].copy()
    competitionGroups = HeatTimes.groupby("Competition_ID")
    for name, group in competitionGroups:
        Averages = Averages.append({"Competition_ID":str(name),"HeatTop9Avg":group.nsmallest(9,"Total Time")["Total Time"].mean()},ignore_index=True)

    Averages["Competition_ID"] = Averages["Competition_ID"].astype('int64')
    df = pd.merge(df,Averages,on="Competition_ID")
    return df

In [25]:
#Trying to gues the weather conditions in a Race 
    #-1:Fast Conditions
    #0: Neutral Conditions
    #1: Slow Condtions
def createConditions(df):
    #Take only the first 3 boats in each race
    data = df[(df["Final Rank"] < 4)]
    #Group them by Place and take q1,q2,q3 of the total times
    q1groupes = data.groupby("Place")["Total Time"].quantile(0.25).to_frame(name="q1")
    q2groupes = data.groupby("Place")["Total Time"].quantile().to_frame(name="q2")
    q3groupes = data.groupby("Place")["Total Time"].quantile(0.75).to_frame(name="q3")
    mergedq1q2 = pd.merge(q1groupes,q2groupes,on="Place")
    merge = pd.merge(mergedq1q2,q3groupes,on="Place").reset_index()

    
    for index, row in df.iterrows():
        #get calculated quantile values for each place
        q1 = merge.loc[merge["Place"] == row["Place"]]["q1"].values[0]
        q2 = merge.loc[merge["Place"] == row["Place"]]["q2"].values[0]
        q3 = merge.loc[merge["Place"] == row["Place"]]["q3"].values[0]

        #if the Top3avg of a race is faster then q1 of that place -> fast condition
        if(row["Top3Avg"] < q1) : df.at[index,"Condition"] = "-1"
        #if the Top3avg of a race is slower then q3 of that place -> slow condition
        elif(row["Top3Avg"] > q3) : df.at[index,"Condition"] = "1"
        #else -> neutral condition
        else: df.at[index,"Condition"] = "0"
    return df
    
   
    

In [34]:
def createMovingWindow(dataset):
    grouped = dataset.groupby("Team")
    results = pd.DataFrame(columns=["Date","Team","Total Time","Final Rank"]).set_index('Date')
    for name, group in grouped:
        df = group[["Date","Team","Total Time","Final Rank"]].set_index("Date").sort_values(by=["Date"])
        df["TotalTime-1"] = df["Total Time"].shift(1) 
        df["Rank-1"] = df["Final Rank"].shift(1)

        df["TotalTime-2"] = df["Total Time"].shift(2) 
        df["Rank-2"] = df["Final Rank"].shift(2)

        df["TotalTime-3"] = df["Total Time"].shift(3) 
        df["Rank-3"] = df["Final Rank"].shift(3)


        df["TotalTime-4"] = df["Total Time"].shift(4) 
        df["Rank-4"] = df["Final Rank"].shift(4)

        df["TotalTime-5"] = df["Total Time"].shift(5) 
        df["Rank-5"] = df["Final Rank"].shift(5)

        results = pd.concat([results,df],sort=False)

    df = pd.merge(dataset,results,on="Team")
    return df

In [35]:
df = pd.read_csv("data.csv",sep=";",error_bad_lines=False)

# Datum opschonen
df = cleanColumns(df)
#df = createRaceIndex(df)
# Datum opsplitens in verschillende columns
df = splitDateInColumns(df)
#Create Teams
df = createTeams(df)
# Convert alle tijden naar seconden
convertTimes(df)

df = createIDs(df)

#Make columns of categorial columns
df = createCompetitionColumns(df)
df = create_Reached_FinalA(df)
df = create_Reached_Semi(df)
df = create_DeltaTimes(df)



df = create_count_FinalA_currentSeason(df)
df = createPb_TimecurrentSeason(df)


df =createAVGTIMETOP3(df)
#getAverageTimeofRace(df)
#df = dropColumns(df)
df = createCompetitionTotal(df)
df = getTotalAFinalsTeam(df)
df = getTotalSemiFinalsTeam(df)
df = createFastestHeatTimeInCompetition(df)
df = createFastestSemiTimeInCompetition(df)
df = createFastestFinalTimeInCompetition(df)
#df = createConditions(df)
df = createFastestTimeInCompetition(df)
#df = createAvgTop3HeatTimeInCompetition(df)




df = createAvgTop3HeatTimeInCompetition(df)
df = createAvgTop3SemiTimeInCompetition(df)

df = createAvgTop9HeatTimeInCompetition(df)
df = createAvgTop9SemiTimeInCompetition(df)
df = rankeRaces(df)
df = rankeRaces250(df)
df = rankeRaces500(df)
df = rankeRaces750(df)
df = createMovingWindow(df)
print(df.columns)

df.to_csv("K2_MEN_DATASET.csv")




C:\Users\Bram Sikkens\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Bram Sikkens\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
Index(['Date', 'Place', 'Time', 'Category', 'Sex', 'Country', 'Lane',
       'Person 1', 'Person 2', 'split time 1', 'split time 2', 'split time 3',
       'Total Time_x', 'Final Rank_x', 'Air Temp', 'Wind Speed',
       'Wind Direction', 'Day', 'Month', 'Year', 'Team',

In [37]:
Race_Dataset = df[['Race_ID','Competition_ID','Date',"Competition Round_FINAL A","Competition Round_FINAL B","Competition Round_FINAL C","Competition Round_HEAT","Competition Round_SEMIFINAL"]].copy().drop_duplicates().set_index("Race_ID").to_csv("Data/Race_Dataset.csv")
Race_Team_Dataset = df[["Race_ID","Team_ID","Country","Lane","Final Rank","split time 1","split time 2","split time 3","Total Time"]].copy().drop_duplicates().set_index(["Race_ID","Team_ID"]).to_csv("Data/Race_Team_Dataset.csv")
Competition_DataSet = df[['Competition_ID','Place','Competiton Type_WORLDCHAMPIONSHIPS',"Competiton Type_WORLDCUP","Year"]].copy().drop_duplicates().set_index("Competition_ID").to_csv("Data/Competition_Dataset.csv")
Team_Dataset = df[["Team_ID","Country","Person 1","Person 2"]].copy().drop_duplicates().set_index("Team_ID").to_csv("Data/Team_Dataset.csv")


Competetition_Analasys_Dataset = df[['Competition_ID','N1THeatCompetition','N1TSemiCompetition',"N1TFinalACompetition","N1TCompetition","HeatTop3Avg","SemiTop3Avg","HeatTop9Avg","SemiTop9Avg"]].copy().drop_duplicates().set_index("Competition_ID").to_csv("Data/Competition_Analasis_Dataset.csv")
Race_Analasis_Dataset = df[["Race_ID","Condition",'TT1', 'TT2', 'TT3', 'TT4', 'TT5', 'TT6', 'TT7', 'TT8',
       'TT9', 'T1_250', 'T2_250', 'T3_250', 'T4_250', 'T5_250', 'T6_250',
       'T7_250', 'T8_250', 'T9_250', 'T1_500', 'T2_500', 'T3_500', 'T4_500',
       'T5_500', 'T6_500', 'T7_500', 'T8_500', 'T9_500', 'T1_750', 'T2_750',
       'T3_750', 'T4_750', 'T5_750', 'T6_750', 'T7_750', 'T8_750', 'T9_750']].copy().set_index("Race_ID").drop_duplicates().to_csv("Data/Race_Analasis_Dataset.csv")
df.to_csv("K2_MEN_DATASET.csv")
Team_Analasis_Dataset = df[["Team_ID","Date","FinalA_count_Season","TotalSemiFinalsTogether","BestSeasonTime","TotalRacesTogether"]].set_index(["Team_ID","Date"]).to_csv("Data/Team_Analasis_Dataset.csv")
Team_Competition_Analasys_Dataset = df[["Team_ID","Competition_ID","Reached_FINAL_A","Reached_SEMI"]]

KeyError: "['Total Time', 'Final Rank'] not in index"